In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain_community.llms import Ollama

# Load PDF
loader = PyPDFLoader(r"C:\Users\karan\Documents\Projects\Gmail\nvidia-ai-aerial-faq.pdf")
documents = loader.load()

# Split text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=7500,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(documents)

# Create embeddings using Ollama
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Create vector store
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings
)

# Initialize Ollama model
llm = Ollama(model="llama3.2:latest")

# Create QA chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever()
)

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="vectorstore_directory"  # Specify directory here
)

# Save to disk
vectorstore.persist()

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain_community.llms import Ollama

# Load the saved vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma(
    persist_directory="vectorstore_directory",
    embedding_function=embeddings
)

# Initialize Ollama
llm = Ollama(model="llama3.2:latest")

# Create the QA chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever()
)

# Query the document
chat_history = []
query = "What is this document about?"
result = qa_chain({"question": query, "chat_history": chat_history})
print(result['answer'])

# For follow-up questions, update chat history
chat_history.append((query, result['answer']))


In [6]:
import os
from cerebras.cloud.sdk import Cerebras
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.llms.base import LLM
from typing import Any, Optional, List
from pydantic import PrivateAttr

# Initialize Cerebras client
client = Cerebras(
    api_key=os.getenv("CEREBRAS_API_KEY", "csk-e2e8kypw838rwmpjxd9nx2vn5jrertm339fnrcnt9c6p8hmx"),
)



class CloudLLM(LLM):
    _client: Any = PrivateAttr()
    model_name: str

    def __init__(self, client: Any, model_name: str):
        super().__init__(model_name=model_name)
        self._client = client

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self._client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=self.model_name,
        )
        return response.choices[0].message.content


    @property
    def _identifying_params(self):
        return {"model_name": self.model_name}

    @property
    def _llm_type(self):
        return "cloud_llm"


# Load the saved vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma(
    persist_directory="vectorstore_directory",
    embedding_function=embeddings
)

# Initialize the custom cloud LLM
cloud_llm = CloudLLM(client=client, model_name="llama3.1-8b")

# Create the conversational retrieval chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=cloud_llm,
    retriever=vectorstore.as_retriever()
)

# Query the document
chat_history = []
query = "What is this document about?"
result = qa_chain({"question": query, "chat_history": chat_history})
print(result['answer'])

# Update chat history for follow-up questions
chat_history.append((query, result['answer']))


This document is about NVIDIA AI Aerial FAQ. It's a set of Questions and Answers regarding the NVIDIA AI Aerial system, its deployment, and various use cases, including ARC-OTA and OAI stack.


In [8]:
import os
import pickle
import base64
import json
from email.mime.text import MIMEText
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from cerebras.cloud.sdk import Cerebras
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.llms.base import LLM
from typing import Any, Optional, List
from pydantic import PrivateAttr

# -------------------- Gmail API Setup --------------------

# Define the scope for Gmail API
SCOPES = [
    'https://www.googleapis.com/auth/gmail.readonly',
    'https://www.googleapis.com/auth/gmail.send'
]

def authenticate_gmail():
    """Authenticate the user and return the Gmail service."""
    creds = None
    # Load credentials from token.pickle if it exists
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If no valid credentials, prompt the user to log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0, access_type='offline', prompt='consent')
        # Save the credentials for future use
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    # Build the Gmail service
    service = build('gmail', 'v1', credentials=creds)
    return service

def get_latest_email(service):
    """Retrieve the most recent email from the user's Primary inbox."""
    try:
        # Fetch the latest email from the Primary category
        results = service.users().messages().list(
            userId='me',
            maxResults=1,
            q="category:primary"
        ).execute()
        messages = results.get('messages', [])
        if not messages:
            print("No messages found in the Primary category.")
            return None
        # Get the message details
        message = service.users().messages().get(
            userId='me',
            id=messages[0]['id'],
            format='full'
        ).execute()
        return message
    except Exception as error:
        print(f"An error occurred while fetching the latest email: {error}")
        return None

def decode_message_body(encoded_body):
    """Decode the base64url-encoded message body."""
    try:
        # Add padding if necessary
        padding = 4 - (len(encoded_body) % 4)
        if padding:
            encoded_body += "=" * padding
        decoded_bytes = base64.urlsafe_b64decode(encoded_body)
        return decoded_bytes.decode('utf-8')
    except Exception as e:
        print(f"Error decoding message body: {e}")
        return ""

def extract_email_details(message):
    """Extract details from the email message."""
    headers = message['payload'].get('headers', [])
    subject = next((header['value'] for header in headers if header['name'] == 'Subject'), "No Subject")
    sender = next((header['value'] for header in headers if header['name'] == 'From'), "Unknown Sender")
    recipient = next((header['value'] for header in headers if header['name'] == 'To'), "Unknown Recipient")
    body = ""
    if 'parts' in message['payload']:
        for part in message['payload']['parts']:
            if part['mimeType'] == 'text/plain' and 'data' in part['body']:
                body = decode_message_body(part['body']['data'])
                break
    elif 'body' in message['payload'] and 'data' in message['payload']['body']:
        body = decode_message_body(message['payload']['body']['data'])
    return {
        "Subject": subject,
        "From": sender,
        "To": recipient,
        "Body": body
    }

def create_reply_message(service, original_message, reply_body):
    """Create a reply message for the given original email."""
    try:
        # Extract necessary details from the original message
        thread_id = original_message['threadId']
        message_id = None
        headers = original_message['payload'].get('headers', [])
        for header in headers:
            if header['name'] == 'Message-ID':
                message_id = header['value']
            elif header['name'] == 'From':
                sender = header['value']
            elif header['name'] == 'To':
                recipient = header['value']
            elif header['name'] == 'Subject':
                subject = header['value']

        # Create the reply email
        reply = MIMEText(reply_body)
        reply['To'] = sender  # Reply to the original sender
        reply['From'] = recipient  # Your email address
        reply['Subject'] = f"Re: {subject}"
        reply['In-Reply-To'] = message_id
        reply['References'] = message_id

        # Encode the message
        raw_message = base64.urlsafe_b64encode(reply.as_bytes()).decode()

        # Create the message object with threadId
        return {
            'raw': raw_message,
            'threadId': thread_id
        }
    except Exception as error:
        print(f"An error occurred while creating the reply message: {error}")
        return None

def send_reply(service, original_message, reply_body):
    """Send a reply to an email message."""
    try:
        reply = create_reply_message(service, original_message, reply_body)
        if reply:
            sent_message = service.users().messages().send(userId='me', body=reply).execute()
            print(f"Reply sent to thread ID: {sent_message['threadId']}")
        else:
            print("Failed to create reply message.")
    except Exception as error:
        print(f"An error occurred while sending the reply: {error}")

# -------------------- Cerebras API Setup --------------------

# Initialize the Cerebras client
CEREBRAS_API_KEY = os.getenv("CEREBRAS_API_KEY", "csk-e2e8kypw838rwmpjxd9nx2vn5jrertm339fnrcnt9c6p8hmx")  # Replace with your actual API key or set as environment variable
client = Cerebras(api_key=CEREBRAS_API_KEY)

def classify_email_body(email_body):
    """Classify the email body as 'FAQ', 'Inquiry', or 'Other'."""
    try:
        # Define the system prompt
        system_prompt = """You are an email classifier. Classify the following email body into one of the following categories: 'FAQ', 'Inquiry', or 'Other'. Respond with the classification only."""
        
        # Create the completion request
        completion = client.chat.completions.create(
            model="llama-3.3-70b",  # Specify the model you wish to use
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": email_body}
            ],
            temperature=0.0,
            max_tokens=10,
            top_p=1,
            stream=False,
            stop=None,
        )

        # Extract and return the classification
        parsed_content = completion.choices[0].message.content.strip()
        return parsed_content

    except Exception as e:
        print(f"Error classifying email body: {e}")
        return "Error"

# -------------------- RAG (Retrieval-Augmented Generation) Setup --------------------

class CloudLLM(LLM):
    _client: Any = PrivateAttr()
    model_name: str

    def __init__(self, client: Any, model_name: str):
        super().__init__(model_name=model_name)
        self._client = client

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        try:
            response = self._client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model=self.model_name,
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"Error generating response with LLM: {e}")
            return "I'm sorry, I couldn't process your request at this time."

    @property
    def _identifying_params(self):
        return {"model_name": self.model_name}

    @property
    def _llm_type(self):
        return "cloud_llm"

def setup_rag():
    """Initialize the RAG system."""
    try:
        # Load the saved vector store
        embeddings = OllamaEmbeddings(model="nomic-embed-text")
        vectorstore = Chroma(
            persist_directory="vectorstore_directory",  # Ensure this directory exists and is populated
            embedding_function=embeddings
        )

        # Initialize the custom cloud LLM
        cloud_llm = CloudLLM(client=client, model_name="llama3.1-8b")  # Ensure the model name is correct and available

        # Create the conversational retrieval chain
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=cloud_llm,
            retriever=vectorstore.as_retriever()
        )

        return qa_chain

    except Exception as e:
        print(f"Error setting up RAG system: {e}")
        return None

def generate_rag_response(qa_chain, query, chat_history=None):
    """Generate a response using the RAG system."""
    try:
        if chat_history is None:
            chat_history = []
        result = qa_chain({"question": query, "chat_history": chat_history})
        return result['answer']
    except Exception as e:
        print(f"Error generating RAG response: {e}")
        return "I'm sorry, I couldn't find the information you're looking for."

# -------------------- Main Workflow --------------------

def main():
    # Step 1: Authenticate and build the Gmail service
    service = authenticate_gmail()

    # Step 2: Fetch the latest email
    latest_email = get_latest_email(service)
    if not latest_email:
        return

    # Step 3: Extract email details
    email_details = extract_email_details(latest_email)
    print("Latest Email Details:")
    print(json.dumps(email_details, indent=4))

    # Step 4: Classify the email body
    email_body = email_details.get('Body', '')
    if not email_body:
        print("No body content to classify.")
        classification = 'No Body Content'
    else:
        classification = classify_email_body(email_body)
        print(f"Classification: {classification}")

    # Step 5: Update email details with classification
    email_details['Classification'] = classification
    # Example: Save to 'classified_email.json'
    with open('classified_email.json', 'w') as f:
        json.dump(email_details, f, indent=4)
    print("Email classification saved to 'classified_email.json'.")

    # Step 6: Generate reply based on classification
    if classification in ['FAQ', 'Inquiry']:
        # Initialize RAG system
        qa_chain = setup_rag()
        if qa_chain:
            # Use the email body as the query
            rag_response = generate_rag_response(qa_chain, email_body)
            reply_body = rag_response
            print(f"Generated RAG Response: {reply_body}")
        else:
            reply_body = "Thank you for your email. We will review your message and respond accordingly."
            print("RAG system not available. Using generic response.")
    else:
        # For 'Other' or 'No Body Content', use predefined responses
        if classification == 'No Body Content':
            reply_body = "Thank you for your email. It appears there was no content in your message. Please provide more details so we can assist you better."
        else:
            reply_body = "Thank you for your email. We will review your message and respond accordingly."

    # Step 7: Send the reply
    send_reply(service, latest_email, reply_body)

if __name__ == '__main__':
    main()

Latest Email Details:
{
    "Subject": "Test Inquiry",
    "From": "Karan Vora <kv2154@nyu.edu>",
    "To": "Karan Vora <karanvora210@gmail.com>",
    "Body": "I want to inquiry about the specifications of the Nvidia Ariel Products and\r\nits usecases\r\n\r\nBest Regards,\r\nKaran Vora\r\nNew York University Class of 2024\r\nMS Computer Engineering\r\nkv2154@nyu.edu\r\nkaranvora210@gmail.com\r\nContact Number: +1 929-930-0027\r\nLinkedin <https://www.linkedin.com/in/karan-vora-574961188/>\r\n"
}
Classification: Inquiry
Email classification saved to 'classified_email.json'.
Generated RAG Response: Subject: Inquiry about NVIDIA AI Aerial Specifications and Usecases

Dear Karan Vora,

Thank you for reaching out to us about NVIDIA AI Aerial Specifications and Usecases. 

To address your inquiry, we'll provide relevant information based on the NVIDIA AI Aerial FAQ document. 

1. **NVIDIA AI Aerial Components, Interfaces, and Usecases:**
   - cuPHY (PHY components) and cuMAC (MAC components)